In [ ]:
library(tidyverse)
library(magrittr)
library(olsrr)

# Multiple Regression

Now that we got the hang of conducting and interpreting regression with a single IV, we're going to move on to being able to use many variables to fit a model that best describes how our IVs potentially influence our DV.

With multiple IVs, we're looking at the effect of one of the IVs "holding all other variables constant" or "controlling for all other variables."

We're still finding the best linear relationship between the DV and our IVs.

#### Simple Linear Regression (one IV):
## $y = \beta_0 + \beta_1x_1 + \varepsilon$
<BR>
    
#### Multiple Linear Regression (multiple IVs):

## $y = \beta_0 + \beta_1x_1 + \beta_2x_2 + \beta_3x_3 .... + \beta_ix_i + \varepsilon$

### Why?

1. We can explain more of the variance in y.
2. We can describe the relationship between y and different x variables, *while holding the other IVs constant.*
3. If we know that a certain variable could explain some of the bias in our data, we can include that variable in the model to "control" for that bias.

## Big Brother
Big Brother is a TV show in which groups of adults (typically 16 per season) are locked into a house (studio) for an entire summer (~80-90 days).  They are filmed 24/7 and in addition to 3 televised episodes per week, viewers can also watch 24/7 "live feeds."  The "houseguests" are competing to win a half million dollars.  Each week one houseguest wins the head of household (HOH) competition.  That HOH nominates 2 HGs for eviction.  Those HGs have 1 chance to avoid eviction - the veto ceremony.  6 HGs participate in the veto ceremony - the 2 nominees, the HOH, and 3 other HGs based on random draw/HG's choice.  If the winner of the veto removes one of the nominees from "the block" the HOH nominates another remaining HG.  One of the two nominees is voted out each week during a live show, then the process is repeated on a weekly basis until 2 HGs remain.  A winner is voted on between those 2 remaining HGs, voted for by a jury of evicted HGs.  

I have obtained data on previous HGs from the last 21 seasons of regular US Big Brother, 1 special online season of US Big Brother (OTT), and 2 special shortened celebrity seasons.  This data was collected by Vince Dixon and made available on his github page.  You can see his methodology and his analysis of the data (specifically focused on racial dynamics) at https://vincedixonportfolio.com/2019/08/29/methodology-behind-big-brother-diversity-data-dive/.

We can't use linear regression to predict a winner, that's a yes/no variable, but we can use the data to predict how long a player remains in the season, and if there are any factors that are significantly associated with their success.

Let's load the data directly from github.

In [ ]:
## use read_csv to read the data from the raw content link 
bbdata <- read_csv("https://raw.githubusercontent.com/vdixon3/big-brother-diversity-data/master/big_brother_data.csv")

## take a peek at the format/variables
glimpse(bbdata)

Given what I know bout Big Brother, I'm going to filter out some of our observations.

1. Regular Seasons 1 and 2 are removed because the game was not fully developed with the current rules at that point.  Season 1 had "America Votes" on nominees.  Season 2 had HOH, but not veto competition.
2. BB OTT is going to be removed since the format was different than the traditional seasons and also involved a component of America influencing nominees.
3. There have been a few people who have "self-evicted" or were "ejected" from the season prematurely.  Given this, their tenure in the game was arbitrarily shortened.

In [ ]:
# make a vector of seasons I want to exclude
exclude = c("bbus1", "bbus2", "bbott1")

# use the %in% operator to compare season_code against multiple values. ! operator negates - keep seasons not in the exclude list
bb <- bbdata %>% filter(!(season_code %in% exclude))  %>% ## remove non-standard seasons
        filter(self_evicted == "no" & ejected == "no")  %>% ## remove self-evicted and ejected players
        ## select specific variables to keep
        select(index, first, last, season_code, total_houseguests, total_days, age, gender, race_ethnicity, 
               lgbts, final_eviction_day, total_vetos, total_hoh, total_wins, other_comp_wins, 
               total_nominations, final_placement) ## select columns to keep
summary(bb)

Next we're going to take a look at some of our categorical variables to see if they require any data cleaning.

In [ ]:
## look at race/ethnicity to see how many groups and size of groups
table(bb$race_ethnicity)

## keep white, black, and lump all other categories into other
bb %<>% mutate(race_ethnicity = fct_lump(race_ethnicity, 2))

## look at table after lumping
table(bb$race_ethnicity)

In [ ]:
## look at the lgbt variable to see what the possible values are and the distribution/frequencies
table(bb$lgbts)

## keep non-lgbt and lump all other destinctions into "other"
bb %<>% mutate(lgbts = fct_lump(lgbts, 1))

## confirm our recode
table(bb$lgbts)

In [ ]:
## total_hoh and total_vetos are numbers, but they are stored as character in the dataset - convert to numeric

bb %<>% mutate(total_vetos = as.numeric(total_vetos), total_hoh = as.numeric(total_hoh))

## review the summary again
summary(bb)

One more data cleaning step - The total_days variable on each record indicates the number of days in that player's season.  There is some variation in total days - from as few as 26 (celebrity seasons) to as many as 99 days (more current regular seasons).  In order to standardize our measure of tenure in the game I'm going to divide the player's final eviction day by the total number of days in the season that that person played - obtaining a proportion of the season that player lasted in the game.

In [ ]:
## create new variable that is eviction day divided by total days in season
bb %<>% mutate(tenure = final_eviction_day / total_days)
summary(bb$tenure)

Wait, what?  Why is there a player whose tenure is longer than the season lasted?  Let's inspect:

In [ ]:
## print players with tenure larger than 1 (1 being the entire season)
bb %>% filter(tenure > 1)


<img src="images/metta.jpg" width="400" height="400">

This appears to be a data entry error - Metta World Peace made it to day 20 of CBB1, not day 40.  We can edit this value.


In [ ]:
## correct final_eviction_day for Metta World Peace
bb[bb$first == "Metta",]$final_eviction_day <- 20

# recalculate our 
bb %<>% mutate(tenure = final_eviction_day / total_days)
summary(bb$tenure)

Looks good!  Let's look at some relationships between our numeric variables.

In [ ]:
colnames(bb)

In [ ]:
pairs(bb[c(7,12:16,18)])

In [ ]:
cor(bb[c(7,12:16,18)])

It looks like the variable with the largest correlation with tenure is total_hoh wins.  The positive correlation indicates that as number of HOH wins increases, the player's tenure in the game also increases.  Let's use this variable as an IV in our first model, a simple linear regression model, to predict tenure. Let's look closer at a scatterplot.

In [ ]:
options(repr.plot.width=5, repr.plot.height=4) ## plot size options for Jupyter notebook ONLY

## use ggplot to create a scatterplot with a "smoothed" line defined by the lm.  We can show the se, or CI of our line as well.
bb %>% ggplot(aes(x=total_hoh, y=tenure)) + ## indicate df, x and y variables.
  geom_point()+ ## scatterplot
  geom_smooth(method=lm, se=TRUE) ## method is lm, show CI

## Fitting the Model - Simple Linear Regression, again
Now we'll fit our model, using total HOH wins to predict a player's tenure in the game.

In [ ]:
mod1 <- lm(tenure ~ total_hoh, data=bb) ## here I've saved the resulting model to "mod1"
summary(mod1)

## Interpreting Model Output - One IV
We'll go ahead and interpret our output to see what it tells us about the influence of HOH wins on tenure in the game.

1. Our intercept is 0.52.  Because zero is a possible value of HOH wins, we can interpret this as the mean of tenure (or the predicted/fitted value of tenure) when a player has zero HOH wins.  So on average a player with zero HOH wins makes it halfway through the game.  

2. Our coefficient for HOH wins is 0.15.  This means that an additional HOH win corresponds to a 0.15 increase in tenure in the game.  Remember that the meaning of this is surviving for an additional 15% of the game.  In a modern/recent season that would be the equivalent of 15% of 99 days, or about 14 additional days - adding 2 weeks to the player's tenure.  

3. The p-value for the total HOH coefficient is below an alpha of 0.05, so total HOH is a significant predictor of tenure in the game.  The coefficient is significantly different from 0.

4. Our R-squared is 0.34.  This means that number of HOH wins explains 34% of the variance in tenure.

5. The p-value associated with our F-statistic is significantly below an alpha of 0.05.  This means that the inclusion of our IV in our model significantly improves the prediction of tenure over a null model, or a model with no IVs (just using the mean/intercept to explain the variance).  This also tells us our R-squared is significantly different from zero.

We need to check our assumptions!

## Post Hoc Checks - Model 1

### Normality of Residuals

In [ ]:
## plot() is used with our saved lm output.  the argument which allows us to print only the plot we specify, 
## the second plot is our QQ plot.
plot(mod1, which = 2)

It appears that the distribution of residuals deviates from a normal distribution in the tails - especially the upper tail.

### Influential Outliers

In [ ]:
plot(mod1, which = 5) ## plot 5 is Residuals vs. Leverage

A few of our points are labeled, but none of our observations are outside of the dashed red lines that indicate our threshold for Cook's distance, so these are either outliers that do not have inordinate influence on the line, or points that have potential large leverage but are not outliers.  Out of curiosity, let's see who these players are.

In [ ]:
bb[c(168,181,296),]

In [ ]:
ols_plot_resid_lev(mod1)

Frankie Grande's large number of HOH wins was partially due to the format of the season he played in - for the majority of the season there was a twist in which there were 2 HOHs per week.  While none of these observations are influential outliers, we may want to remove the entirety of Season 16 in future analyses.

### Homoscedasticity
Let's check to see if our model violates the assumption of constant variance.

In [ ]:
ols_test_breusch_pagan(mod1)

We reject the null hypothesis and therefore conclude the variance is not constant.  However, we know this test is pretty sensitive.  Let's look at our plot of residuals vs. fitted values.  This plot will also tell us about if our ...

### Errors are Independent

In [ ]:
plot(mod1, which = 1) ## residuals vs. fitted is plot #1

There is definitely a "cone" or "funnel" shape, so I'd conclude we are violating the assumption of homoscedasticity.  There also appears to be a slight curved relationship between our residuals and fitted, indicating that there is perhaps other variables that are related to tenure that we're not yet accounting for in our model.  Good thing that we're getting ready to add more variables!  First let's remove our non-standard season, season 16 and refit our simple model to see if our results change any.

In [ ]:
## save a subset of the data without season 16
bb_sub16 <- bb %>% filter(season_code != "bbus16")

## refit model one with the data without season 16
mod1b <- lm(tenure ~ total_hoh, data=bb_sub16) 
summary(mod1b)

It doesn't appear to have a huge impact on the model, so we'll just proceed without removing season 16.  Either decision (to keep or remove) could be defended by the analyst - it's one of those places where the analyst has to 1. make a choice 2. clearly report that they made this decision and any potential impact that could have on their analysis. Could it bias the analysis to a point where it would limit inference to the population /generalizability?

## Model 2 - More variables!
Time to add more variables.  Let's add another numerical variable, this time the number of nominations.  Nominations are related to be selected for eviction, so definitely could influence tenure.  Let's see how it affects our model.

In [ ]:
mod2 <- lm(tenure ~ total_hoh + total_nominations, data=bb) 
summary(mod2)

## Model 2 - Interpretation
It's interpretation time again! This time we have two slope coefficients!

1. Our intercept is now 0.37.  Why did it change?  It's because this is now the mean of y (fitted value of y) when both hoh wins AND nominations are zero.  

2. The coefficients for HOH wins has changed too!  Why is this?  It's because now we interpret this coefficient as:
**An additional HOH win corresponds to a 12% increase in game tenure....**
    - ...holding all other variables in the model constant.
    - ...holding all other variables constant.
    - ...all else equal.
    - ...accounting for other variables in the model.
    - ...when controlling for `total_nominations`.
    - ...holding all other variables fixed.
    
    This means that the coefficient reflects the sole influence of HOH wins separated from the influence of the other variables. 
    HOH wins remains a significant predictor of tenure - the p-value is below alpha = 0.05 and therefore we reject the null hypothesis - the coefficient is significantly different from zero.
   
3. The coefficient for nominations is also significantly different from zero, indicating that nominations is also a significant predictor of tenure.  It's not in the direction you might expect given that nominations lead to eviction which ends tenure.  However, the longer a player is in the game the more chances they have to be nominated.  

    Our interpretation of the coefficient is:  **Holding all other variables equal, each additional nomination corresponds to a 7% increase in tenure in the game.**
    
4. Our R-squared is now 0.49.  Remember this reflects the amount of variance explained by the entire model, and not any single IV.  So overall, with both of our IVs, the model is explaining almost 50% of the variance in tenure.  That's not bad at all!

5. The overall F-test and related p-value.  This test is also an "omnibus" test of the entire model - is the model with both of our IVs better than a null model with no predictors?  We reject the null hypothesis, therefore our model is significantly better than a null model.

But what if we wanted to know if this model is better than model 1?  Can we do that?

### Model Comparison
We can compare models to see if one model is a significant improvement over a previous model.  In order to do this, our reduced/smaller model needs to be **nested** inside our larger model.  This means that the smaller model needs to include a subset of the variables in the larger model and cannot include any variables that are not in the larger model.

In our case our second model contains the IV from the first model, HOH wins, and an additional IV.  Let's see if this model is significantly better than the previous model.  

The way we do this is through an F-test, but this time instead of comparing our model to a null model, we're instead comparing our "full" (larger) model to a "reduced" (smaller) model.

In [ ]:
## we use the anova() NOT aov() function and pass it our two saved models.
anova(mod1, mod2)

What does this mean?  The degrees of freedom of 1 indicates that our full model has 1 more parameter (IV) than our reduced model.  But the most important part is the p-value.  For this test our null hypothesis is that the models fit equally as well; the alternative hypotheis is that the fuller model is a significant improvement over the smaller model.  In this case our p-value is below alpha = 0.05, therefore our model with 2 IVs is a significant improvement over our model with only one IV.

Why do we care?

### Parsimony
When we construct statistical models we want to obtain the best fitting model that uses the fewest variables / assumptions.  We want to use the simplest model that is adequate to explain our data. There is no benefit to including more predictors that do not significantly improve the model.

## Model 2 - Post Hoc Checks

In [ ]:
## NORMALITY OF RESIDUALS
ols_plot_resid_qq(mod2) ## use QQ plot function from olsrr package

Our residuals seem normally distributed for the most part, however we do deviate from normality in the tails - this time it seems like the largest deviation from normality is in the lower tail.

In [ ]:
## INFLUENTIAL OUTLIERS
ols_plot_resid_lev(mod2)

In [ ]:
bb[c(163,179,181,196),]

We do have outliers and influential observations, but again none are both influential and outliers.

In [ ]:
## RESID VS. FITTED -- HOMOSCEDASTICITY AND INDEPENDENT ERRORS
ols_plot_resid_fit(mod2)

It looks like our heteroscedasticity is worse, and there may be a slight curvilinear relationship between our residuals and fitted values.

## Model 3 - Adding a Categorical Predictor
We have two numerical IVs, but what happens when we add a categorical variable?  Let's add gender - which is a categorical variable with only two levels.  Remember this will be "dummy" coded - it will be included in the model as a 0/1 variable -- an "on/off" switch.  By default R uses the lowest indexed level as the "reference group" to which the other levels of the categorical variable are compared.

In [ ]:
mod3 <- lm(tenure ~ total_hoh + total_nominations + gender, data=bb) 
summary(mod3)

Before we interpret the model, let's see if the model is a significant improvement over model 2.

In [ ]:
## we use the anova() NOT aov() function and pass it our two saved models.
anova(mod2, mod3)

This model is not a significant improvement over model 2 without gender - the p-value is above 0.05.  Therefore, in the interest of parsimony, we will not include gender in the model.  What about race?

In [ ]:
mod4 <- lm(tenure ~ total_hoh + total_nominations + race_ethnicity, data=bb) 
summary(mod4)

In [ ]:
anova(mod2, mod4)

Again, adding race/ethnicity does not significantly improve the model.

Maybe we were not destined to add a categorical variable here.  Before we move on, maybe we'll try to add age to the model (another numerical variable).

### THREE VARIABLES ?!?!?!

In [ ]:
mod5 <- lm(tenure ~ total_hoh + total_nominations + age, data=bb) 
summary(mod5)

In [ ]:
anova(mod2, mod5)

### Model Interpretation

The inclusion of age as a predictor made a significant improvement to the model - the p-value for the test of the full vs. reduced model indicates that we can reject null and conclude that the added variable improves the model.

Quick interpretations!

1. The intercept is no longer meaningful - since age can't equal 0.

2. HOH wins is a significant predictor of tenure.  Holding all other variables constant, each HOH win is associated with an additional 13% increase (13% of the season) in tenure.

3. Number of nominations is a significant predictor of tenure.  Holding all other variables constant, each additional nomination is associated with an increase in tenure of 7% of the season length.

4. Age is a significant predictor of tenure, but has a small unstandardized effect size.  An increase in one year of age is associated with an increase in tenure of 0.4% of the season (half of a day in a standard season), holding all else equal.

5. All of our IVs explain 51% of the variance in tenure.

6. Our model is significantly better than a null model (which is not news - this model is significantly better than a nested model, which is fuller than the null model).

### Post Hoc Checks
Quick Check of our plots!

In [ ]:
## NORMALITY OF RESIDUALS
ols_plot_resid_qq(mod5) ## use QQ plot function from olsrr package
## INFLUENTIAL OUTLIERS
ols_plot_resid_lev(mod5)
## RESID VS. FITTED -- HOMOSCEDASTICITY AND INDEPENDENT ERRORS
ols_plot_resid_fit(mod5)

Are our residuals slightly more normal in the tails than before?  I'm not sure, but it looks like it might be.  

We have an influential outlier now, observation 225.  Additionally there is one non-outlier with a seemingly large amount of leverage, observation 97.  Given our inspection of the data below, these appear to be older players - one with a decent amount of comp wins and a few nominations.

In [ ]:
bb[c(97,225),]

## Using Regression Models for Prediction?
As I mentioned in the last lecture, typically academics and social scientists use regression for description of relationships between variables and to uncover potential influences of the outcomes they study.  However, in data science, regression models can be used for prediction.  What if we wanted to use only the information we had about the players before the game - age, gender, and race/ethnicity - and see if we could predict the tenure of future players.  

When we test models for prediction we typically split our data into train/test splits, where the training data is used to fit the model.  Then the model is used to make predictions for the test data, and those predicted values are compared to the actual observed values.  We can do this two ways:

1. Randomly sample ~75-80% of the observations for training, using the rest for the testing.
2. Use the element of time - given that we want to predict future season outcomes using the past seasons, and presumably we could improve our model between seasons to predict the next season, we could hold the data from the last two seasons for our test set, training the model on all earlier data.

In [ ]:
bb_sub <- bb %>% filter(!season_code %in% c("cbbus1", "cbbus2"))

bb_test <- bb_sub %>% filter(season_code %in% c("bbus20", "bbus21"))
bb_train <- bb_sub %>% filter(!season_code %in% c("bbus20", "bbus21"))

In [ ]:
dim(bb_sub)
dim(bb_test)
dim(bb_train)

In [ ]:
# fit our model to our training data with only the information we have preseason
bbmod <- lm(tenure ~ age + race_ethnicity + gender, data = bb_train) # using bb_train only
summary(bbmod)

In [ ]:
preds <- predict(bbmod, bb_test) ## create predictions for "fresh" test data not used to fit the model
bb_test <- cbind(bb_test, preds)

In [ ]:
## compare predicted values to actual observations
ggplot(aes(x=tenure, y=preds), data = bb_test) + ## indicate df, x and y variables.
  geom_point() + 
  expand_limits(x = 0, y = 0) +
  geom_abline(slope = 1, intercept = 0)